In [1]:
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.applications.inception_v3 import preprocess_input as inception_preprocess_input

from keras.applications.xception import Xception,preprocess_input
from keras.applications.xception import preprocess_input as xception_preprocess_input

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg16 import preprocess_input as vgg16_preprocess_input


from keras.models import Sequential, Model,load_model
from keras.preprocessing import image

from keras.wrappers.scikit_learn import KerasRegressor



import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from PIL import Image
import glob




/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model_inception =load_model("../models/InceptionResNet_Finetuning/inceptionResNet_finetuning_train10.h5")

model_xception =load_model("../models/Xception/xception_finetuning_train10.h5")

model_vgg16 =load_model("../models/VGG_Finetuning/vgg16_finetuning_train10.h5")




In [5]:
img_width_xception_inception, img_height_xception_inception = 299, 299
img_width_vgg, img_height_vgg = 224, 224




In [6]:

def loadAllImage(img_name_list, preprocess_input,img_width,img_height):

  imageList = []
  for imgName in img_name_list:
    img = image.load_img(imgName, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    imageList.append(x)
  return np.asarray(imageList)





In [7]:
#dog_img_name_list = glob.glob('./' + "../original_data/dog" + '/*.' + "jpg") + glob.glob('./' + "../data/validation/dogs" + '/*.' + "jpg")
#cat_img_name_list = glob.glob('./' + "../original_data/cat" + '/*.' + "jpg") + glob.glob('./' + "../data/validation/cats" + '/*.' + "jpg")

dog_img_name_list = glob.glob('./' + "../data/train1/dogs" + '/*.' + "jpg")
cat_img_name_list = glob.glob('./' + "../data/train1/cats" + '/*.' + "jpg")


inception_dog_images = loadAllImage(dog_img_name_list, inception_preprocess_input,img_width_xception_inception,img_height_xception_inception)
inception_cat_images = loadAllImage(cat_img_name_list, inception_preprocess_input,img_width_xception_inception,img_height_xception_inception)

xception_dog_images = loadAllImage(dog_img_name_list, xception_preprocess_input,img_width_xception_inception,img_height_xception_inception)
xception_cat_images = loadAllImage(cat_img_name_list, xception_preprocess_input,img_width_xception_inception,img_height_xception_inception)

vgg_dog_images = loadAllImage(dog_img_name_list, vgg16_preprocess_input,img_width_vgg,img_height_vgg)
vgg_cat_images = loadAllImage(cat_img_name_list, vgg16_preprocess_input,img_width_vgg,img_height_vgg)



catLabels =  np.array([[0] for i in range(0,len(cat_img_name_list))])
dogLabels =  np.array([[1] for i in range(0,len(dog_img_name_list))])

In [8]:
inception_catPredictedList = []
for index in range(0,len(inception_cat_images)):
    predictedResult= model_inception.predict(np.array([inception_cat_images[index]]))
    inception_catPredictedList.append((predictedResult[0][0],cat_img_name_list[index]))


inception_dogPredictedList = []
for index in range(0,len(inception_dog_images)):
    predictedResult= model_inception.predict(np.array([inception_dog_images[index]]))
    inception_dogPredictedList.append((predictedResult[0][0],dog_img_name_list[index]))


In [9]:
xception_catPredictedList = []
for index in range(0,len(inception_cat_images)):
  predictedResult= model_xception.predict(np.array([xception_cat_images[index]]))
  xception_catPredictedList.append((predictedResult[0][0],cat_img_name_list[index]))


xception_dogPredictedList = []
for index in range(0,len(inception_dog_images)):
  predictedResult= model_xception.predict(np.array([xception_dog_images[index]]))
  xception_dogPredictedList.append((predictedResult[0][0],dog_img_name_list[index]))



In [10]:
vgg_catPredictedList = []
for index in range(0,len(inception_cat_images)):
  predictedResult= model_vgg16.predict(np.array([vgg_cat_images[index]]))
  vgg_catPredictedList.append((predictedResult[0][0],cat_img_name_list[index]))


vgg_dogPredictedList = []
for index in range(0,len(inception_dog_images)):
  predictedResult= model_vgg16.predict(np.array([vgg_dog_images[index]]))
  vgg_dogPredictedList.append((predictedResult[0][0],dog_img_name_list[index]))





In [12]:
inception_result = []
for tmp in np.concatenate([inception_catPredictedList,inception_dogPredictedList]):
    inception_result.append(float(tmp[0]))
xception_result = []
for tmp in np.concatenate([xception_catPredictedList,xception_dogPredictedList]):
    xception_result.append(float(tmp[0]))
vgg_result = []
for tmp in np.concatenate([vgg_catPredictedList,vgg_dogPredictedList]):
    vgg_result.append(float(tmp[0]))
    
inception_result = np.array(inception_result)
xception_result = np.array(xception_result)
vgg_result = np.array(vgg_result)

In [14]:
labelList = []

for tmp in catLabels:
    labelList.append(0)
for tmp in dogLabels:
    labelList.append(1)
    
labelList = np.array(labelList)



In [15]:
X = []
for index in range(0, len(labelList)):
    X.append(np.array([inception_result[index],xception_result[index],vgg_result[index]]))
X = np.array(X)

In [16]:

from sklearn import linear_model
from sklearn.externals import joblib
clf = linear_model.LinearRegression()

In [17]:
clf.fit(X, labelList)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [18]:
a = clf.coef_
b = clf.intercept_  

# 回帰係数
print("回帰係数:", a)
print("切片:", b) 
print("決定係数(学習用):", clf.score(X, labelList))


回帰係数: [0.32335399 0.08938924 0.588434  ]
切片: -0.0009623017598811012
決定係数(学習用): 0.9989811009118621


In [19]:
joblib.dump(clf, '../models/Ansamble/regression.pkl')
#clf = joblib.load('../models/Ansamble/regression.pkl');

['../models/Ansamble/regression.pkl']